In [1]:
import pandas as pd

In [2]:
hnposts = pd.read_csv("../data/raw/HN_posts_year_to_Sep_26_2016.csv")

In [3]:
hnposts.head()

,id,title,url,num_points,num_comments,author,created_at
0,12579008,You have two days to comment if you want stem ...,http://www.regulations.gov/document?D=FDA-2015...,1,0,altstar,9/26/2016 3:26
1,12579005,SQLAR the SQLite Archiver,https://www.sqlite.org/sqlar/doc/trunk/README.md,1,0,blacksqr,9/26/2016 3:24
2,12578997,What if we just printed a flatscreen televisio...,https://medium.com/vanmoof/our-secrets-out-f21...,1,0,pavel_lishin,9/26/2016 3:19
3,12578989,algorithmic music,http://cacm.acm.org/magazines/2011/7/109891-al...,1,0,poindontcare,9/26/2016 3:16
4,12578979,How the Data Vault Enables the Next-Gen Data W...,https://www.talend.com/blog/2016/05/12/talend-...,1,0,markgainor1,9/26/2016 3:14


In [4]:
hnposts.shape

(293119, 7)

In [5]:
hnposts["title"][0]

'You have two days to comment if you want stem cells to be classified as your own'

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
cv = CountVectorizer(max_df=0.9, min_df=2, stop_words="english")

In [8]:
dtm = cv.fit_transform(hnposts["title"])

In [9]:
dtm

<293119x41917 sparse matrix of type '<class 'numpy.int64'>'
	with 1534334 stored elements in Compressed Sparse Row format>

In [10]:
from sklearn.decomposition import LatentDirichletAllocation

In [11]:
LDA = LatentDirichletAllocation(n_components=8, random_state=42)

In [12]:
LDA.fit(dtm)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=8, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [13]:
len(cv.get_feature_names())
import random

random_word_id = random.randint(0, 41917)
cv.get_feature_names()[random_word_id]

'crank'

In [14]:
len(LDA.components_)
LDA.components_.shape

(8, 41917)

In [15]:
single_topic = LDA.components_[0]

In [16]:
single_topic.argsort()

array([29445, 25497, 27657, ..., 23329, 16027, 24996])

In [17]:
top_fifteen_words = single_topic.argsort()[-15:]
top_fifteen_words

array([32310,  8149, 39866, 35754, 24896, 25878,  5435,  6434, 22318,
           1, 41571, 41576, 23329, 16027, 24996])

In [18]:
for index in top_fifteen_words:
    print(cv.get_feature_names()[index])

san
computer
video
study
net
old
brain
case
man
000
year
years
microsoft
google
new


In [19]:
for index, topic in enumerate(LDA.components_):
    print(f"THE TOP 15 WORDS FOR TOPIC #{index}")
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print("\n\n\n")

THE TOP 15 WORDS FOR TOPIC #0
['san', 'computer', 'video', 'study', 'net', 'old', 'brain', 'case', 'man', '000', 'year', 'years', 'microsoft', 'google', 'new']




THE TOP 15 WORDS FOR TOPIC #1
['cars', 'car', 'driving', 'startup', 'media', 'new', 'deep', 'self', 'social', 'life', 'business', 'machine', 'source', 'learning', 'open']




THE TOP 15 WORDS FOR TOPIC #2
['analysis', 'global', 'visual', 'quantum', 'chrome', 'pdf', 'engineering', 'ai', 'new', 'model', 'music', 'search', 'tesla', 'video', 'google']




THE TOP 15 WORDS FOR TOPIC #3
['apps', 'guide', 'data', '10', 'mobile', 'science', 'android', 'windows', 'free', 'ios', 'linux', 'best', 'app', 'ask', 'hn']




THE TOP 15 WORDS FOR TOPIC #4
['says', 'real', 'iphone', 'time', 'encryption', 'twitter', '2016', 'china', 'startup', 'new', 'facebook', 'big', 'tech', 'apple', 'data']




THE TOP 15 WORDS FOR TOPIC #5
['pdf', 'intelligence', 'learn', 'need', 'real', 'time', 'language', 'things', 'know', 'internet', 'people', 'software

In [20]:
topic_results = LDA.transform(dtm)

In [21]:
topic_results.shape

(293119, 8)

In [22]:
topic_results[0].round(3)

array([0.604, 0.018, 0.018, 0.289, 0.018, 0.018, 0.018, 0.018])

In [23]:
topic_results.argmax(axis=1) + 1

array([1, 7, 1, ..., 5, 7, 2])

In [24]:
hnposts["Topic"] = topic_results.argmax(axis=1)

In [25]:
hnposts.head(50)

,id,title,url,num_points,num_comments,author,created_at,Topic
0,12579008,You have two days to comment if you want stem ...,http://www.regulations.gov/document?D=FDA-2015...,1,0,altstar,9/26/2016 3:26,0
1,12579005,SQLAR the SQLite Archiver,https://www.sqlite.org/sqlar/doc/trunk/README.md,1,0,blacksqr,9/26/2016 3:24,6
2,12578997,What if we just printed a flatscreen televisio...,https://medium.com/vanmoof/our-secrets-out-f21...,1,0,pavel_lishin,9/26/2016 3:19,0
3,12578989,algorithmic music,http://cacm.acm.org/magazines/2011/7/109891-al...,1,0,poindontcare,9/26/2016 3:16,2
4,12578979,How the Data Vault Enables the Next-Gen Data W...,https://www.talend.com/blog/2016/05/12/talend-...,1,0,markgainor1,9/26/2016 3:14,4
5,12578975,Saving the Hassle of Shopping,https://blog.menswr.com/2016/09/07/whats-new-w...,1,1,bdoux,9/26/2016 3:13,5
6,12578954,Macalifa A new open-source music app for UWP ...,http://forums.windowscentral.com/windows-phone...,1,0,thecodrr,9/26/2016 3:06,1
7,12578942,GitHub theweavrs/Macalifa: A music player wri...,https://github.com/theweavrs/Macalifa,1,0,thecodrr,9/26/2016 3:04,5
8,12578919,Google Allo first Impression,http://prodissues.com/2016/09/google-allo-firs...,3,0,jandll,9/26/2016 2:57,0
9,12578918,Advanced Multimedia on the Linux Command Line,https://avi.alkalay.net/2016/09/multimedia-lin...,1,0,mynameislegion,9/26/2016 2:56,5


In [26]:
data = cv.fit_transform(hnposts["title"])
data


<293119x41917 sparse matrix of type '<class 'numpy.int64'>'
	with 1534334 stored elements in Compressed Sparse Row format>

In [27]:
result = LDA.transform(cv.transform(["Six Recipes for Software Managers"]))
result.argmax()

3

In [28]:
def predictCluster(title):
    result = LDA.transform(cv.transform([title]))
    return result.argmax()

In [29]:
predictCluster("Kickstarting a new wave of self driving car companies")

1